### Imports

In [4]:
import os
import pickle
import IPython
import sklearn
import numpy as np
import kmapper as km
import pandas as pd
import seaborn as sns
from umap import UMAP
import tqdm

from sklearn import cluster, preprocessing, manifold, decomposition
from sklearn.model_selection import StratifiedKFold, KFold
from scipy.spatial import distance
from scipy.sparse import issparse
from datetime import datetime
from collections import defaultdict
from datetime import datetime
import inspect
import itertools
import sys
import warnings

from sklearn.manifold import t_sne, isomap

from sklearn import datasets
from sklearn.cluster import DBSCAN

from sklearn.decomposition import TruncatedSVD
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
sns.set(style='white', rc={'figure.figsize':(12,8)})

ModuleNotFoundError: No module named 'umap'

### Some tweaks to kmapper (remove forced index column adding to original data)

In [2]:
class CustomMapper(km.KeplerMapper):
    def map(self,
            projected_X,
            inverse_X=None,
            clusterer=cluster.DBSCAN(eps=0.5, min_samples=3),
            nr_cubes=None,
            overlap_perc=None,
            coverer=km.Cover(nr_cubes=10, overlap_perc=0.1),
            nerve=km.GraphNerve()):
        """Apply Mapper algorithm on this projection and build a simplicial complex. Returns a dictionary with nodes and links.
        Parameters
        ----------
        projected_X : Numpy Array
            Output from fit_transform
        inverse_X : Numpy Array
            Original data. If `None`, then use `projected_X` for clustering.
        clusterer:
            Scikit-learn API compatible clustering algorithm. Default: DBSCAN
        nr_cubes : Int
            The number of intervals/hypercubes to create. Default = 10. (DeprecationWarning: define Cover explicitly in future versions)
        overlap_perc : Float
            The percentage of overlap "between" the intervals/hypercubes. Default = 0.1. (DeprecationWarning: define Cover explicitly in future versions)
        coverer : kmapper.Cover
            Cover scheme for lens. Instance of kmapper.cover providing methods `define_bins` and `find_entries`.
        nerve : kmapper.Nerve
            Nerve builder implementing `__call__(nodes)` API
        Returns
        =======
        simplicial_complex : dict
            A dictionary with "nodes", "links" and "meta" information.
        Example
        =======
        >>> simplicial_complex = mapper.map(projected_X, inverse_X=None, clusterer=cluster.DBSCAN(eps=0.5,min_samples=3),nr_cubes=10, overlap_perc=0.1)
        >>>print(simplicial_complex["nodes"])
        >>>print(simplicial_complex["links"])
        >>>print(simplicial_complex["meta"])
        """

        start = datetime.now()

        nodes = defaultdict(list)
        meta = defaultdict(list)
        graph = {}

        # If inverse image is not provided, we use the projection as the inverse image (suffer projection loss)
        if inverse_X is None:
            inverse_X = projected_X

        if nr_cubes is not None or overlap_perc is not None:
            # If user supplied nr_cubes or overlap_perc,
            # use old defaults instead of new Cover
            nr_cubes = nr_cubes if nr_cubes else 10
            overlap_perc = overlap_perc if overlap_perc else 0.1
            self.coverer = km.Cover(nr_cubes=nr_cubes,
                                 overlap_perc=overlap_perc)

            warnings.warn(
                "Explicitly passing in nr_cubes and overlap_perc will be deprecated in future releases. Please supply Cover object.", DeprecationWarning)
        else:
            self.coverer = coverer

        if self.verbose > 0:
            print("Mapping on data shaped %s using lens shaped %s\n" %
                  (str(inverse_X.shape), str(projected_X.shape)))

        # Prefix'ing the data with ID's
        ids = np.array([x for x in range(projected_X.shape[0])])
        projected_X = np.c_[ids, projected_X]
        inverse_X = inverse_X

        # Cover scheme defines a list of elements
        bins = self.coverer.define_bins(projected_X)

        # Algo's like K-Means, have a set number of clusters. We need this number
        # to adjust for the minimal number of samples inside an interval before
        # we consider clustering or skipping it.
        cluster_params = clusterer.get_params()
        min_cluster_samples = cluster_params.get("n_clusters", 1)

        if self.verbose > 1:
            print("Minimal points in hypercube before clustering: %d" %
                  (min_cluster_samples))

        # Subdivide the projected data X in intervals/hypercubes with overlap
        if self.verbose > 0:
            bins = list(bins)  # extract list from generator
            total_bins = len(bins)
            print("Creating %s hypercubes." % total_bins)

        for i, cube in enumerate(bins):
            # Slice the hypercube:
            #  gather all entries in this element of the cover
            hypercube = self.coverer.find_entries(projected_X, cube)

            if self.verbose > 1:
                print("There are %s points in cube_%s / %s" %
                      (hypercube.shape[0], i, total_bins))

            # If at least min_cluster_samples samples inside the hypercube
            if hypercube.shape[0] >= min_cluster_samples:

                # Cluster the data point(s) in the cube, skipping the id-column
                # Note that we apply clustering on the inverse image (original data samples) that fall inside the cube.
                inverse_x = inverse_X[[int(nn) for nn in hypercube[:, 0]]]

                clusterer.fit(inverse_x)

                if self.verbose > 1:
                    print("Found %s clusters in cube_%s\n" % (
                        np.unique(clusterer.labels_[clusterer.labels_ > -1]).shape[0], i))

                # TODO: I think this loop could be improved by turning inside out:
                #           - partition points according to each cluster
                # Now for every (sample id in cube, predicted cluster label)
                for a in np.c_[hypercube[:, 0], clusterer.labels_]:
                    if a[1] != -1:  # if not predicted as noise

                        # TODO: allow user supplied label
                        #   - where all those extra values necessary?
                        cluster_id = "cube{}_cluster{}".format(i, int(a[1]))

                        # Append the member id's as integers
                        nodes[cluster_id].append(int(a[0]))
                        meta[cluster_id] = {
                            "size": hypercube.shape[0], "coordinates": cube}
            else:
                if self.verbose > 1:
                    print("Cube_%s is empty.\n" % (i))

        links, simplices = nerve(nodes)

        graph["nodes"] = nodes
        graph["links"] = links
        graph["simplices"] = simplices
        graph["meta_data"] = {
            "projection": self.projection if self.projection else "custom",
            "nr_cubes": self.coverer.nr_cubes,
            "overlap_perc": self.coverer.overlap_perc,
            "clusterer": str(clusterer),
            "scaler": str(self.scaler)
        }
        graph["meta_nodes"] = meta

        # Reporting
        if self.verbose > 0:
            self._summary(graph, str(datetime.now() - start))

        return graph

    def _summary(self, graph, time):
        # TODO: this summary is relevant to the type of Nerve being built.
        links = graph["links"]
        nodes = graph["nodes"]
        nr_links = sum(len(v) for k, v in links.items())

        print("\nCreated %s edges and %s nodes in %s." %
              (nr_links, len(nodes), time))

### MetaKMeans class can estimate n_clusters based on a number of observations

In [3]:
class MetaKMeans(sklearn.base.BaseEstimator, sklearn.base.ClusterMixin, sklearn.base.TransformerMixin):
    def __init__(self,
                 init='k-means++',
                 n_init=10,
                 max_iter=300,
                 tol=0.0001,
                 precompute_distances='auto',
                 verbose=0,
                 random_state=None,
                 copy_x=True,
                 n_jobs=1,
                 algorithm='auto'):
        self.init = init
        self.n_init = n_init
        self.max_iter = max_iter
        self.tol = tol
        self.precompute_distances = precompute_distances
        self.verbose = verbose
        self.random_state = random_state
        self.copy_x = copy_x
        self.n_jobs = n_jobs
        self.algorithm = algorithm
        self._temp_clusterer = None
    
    def get_params(self, deep=True):
        params = {
            'init': self.init,
            'n_init': self.n_init,
            'max_iter': self.max_iter,
            'tol': self.tol,
            'precompute_distances': self.precompute_distances,
            'verbose': self.verbose,
            'random_state': self.random_state,
            'copy_x': self.copy_x,
            'n_jobs': self.n_jobs,
            'algorithm': self.algorithm
        }
        return params
    
    def _estimate_n_clusters(self, n):
        return max(1, int(np.log1p(n)) // 2)

    def fit(self, X, y=None):
        n_clusters = self._estimate_n_clusters(len(X))
        self._temp_clusterer = sklearn.cluster.KMeans(n_clusters=n_clusters, **self.get_params())
        self._temp_clusterer.fit(X, y)
        self.labels_ = self._temp_clusterer.labels_.copy()

    def fit_predict(self, X, y=None):
        self.fit(X, y)
        return self.labels_

### Displaying html inside notebook cell

In [6]:
def display(path_html):
    """ Displays a html file inside a Jupyter Notebook output cell.
    
    Parameters
    ----------
    path_html : str
        Path to html. Use file name for file inside current working 
        directory. Use `file://` browser url-format for path to local file.
        Use `https://` urls for externally hosted resources.

    Notes
    -----
    Thanks to https://github.com/smartinsightsfromdata for the issue:
    https://github.com/MLWave/kepler-mapper/issues/10

    """

    iframe = '<iframe src=' + path_html \
            + ' width=100%% height=800 frameBorder="0"></iframe>'
    IPython.core.display.display(IPython.core.display.HTML(iframe))

### Read serialized dataset
* graph_trans_df --- dataframes that contains fraud labels information for each transaction
* baseamount_agg_mean_values --- aggregated features (data where clusteting will take place)
* pca_mean_transformed --- 2d pca of aggregated features
* isof_scores_off_ops --- isolation forest scores for each sample
* mean_colnames_selected --- names for aggregated features

In [5]:
with open('tda_isof_pca_dataset_sample.pkl', 'rb') as handle:
    tda_isof_pca_dataset_sample = pickle.load(handle)

In [12]:
graph_trans_df = tda_isof_pca_dataset_sample['graph_trans_df']
baseamount_agg_mean_values = tda_isof_pca_dataset_sample['baseamount_agg_mean_values']
pca_mean_transformed = tda_isof_pca_dataset_sample['pca_mean_transformed']
isof_scores_off_ops = tda_isof_pca_dataset_sample['isof_scores_off_ops']
mean_colnames_selected = tda_isof_pca_dataset_sample['mean_colnames_selected']

labels = graph_trans_df[('all', 'target')].values

### concatenate pca output and isolation forest scores into a 3d projection

In [8]:
proj_3d = np.c_[pca_mean_transformed, isof_scores_off_ops]

In [ ]:
# Initialize
mapper = CustomMapper(verbose=2)

### Create graph with Mapper algorithm

In [10]:
# Create dictionary called 'graph' with nodes, edges and meta-information
graph = mapper.map(proj_3d, baseamount_agg_mean_values, clusterer=MetaKMeans(n_jobs=10), nr_cubes=15, overlap_perc=0.2)

/home/const.belev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: DeprecationWarning: Explicitly passing in nr_cubes and overlap_perc will be deprecated in future releases. Please supply Cover object.


Mapping on data shaped (821659, 782) using lens shaped (821659, 3)

Minimal points in hypercube before clustering: 1
Creating 3375 hypercubes.
There are 0 points in cube_0 / 3375
Cube_0 is empty.

There are 0 points in cube_1 / 3375
Cube_1 is empty.

There are 0 points in cube_2 / 3375
Cube_2 is empty.

There are 0 points in cube_3 / 3375
Cube_3 is empty.

There are 0 points in cube_4 / 3375
Cube_4 is empty.

There are 0 points in cube_5 / 3375
Cube_5 is empty.

There are 0 points in cube_6 / 3375
Cube_6 is empty.

There are 0 points in cube_7 / 3375
Cube_7 is empty.

There are 0 points in cube_8 / 3375
Cube_8 is empty.

There are 0 points in cube_9 / 3375
Cube_9 is empty.

There are 0 points in cube_10 / 3375
Cube_10 is empty.

There are 0 points in cube_11 / 3375
Cube_11 is empty.

There are 0 points in cube_12 / 3375
Cube_12 is empty.

There are 0 points in cube_13 / 3375
Cube_13 is empty.

There are 0 points in cube_14 / 3375
Cube_14 is empty.

There are 0 points in cube_15 / 3375


There are 0 points in cube_140 / 3375
Cube_140 is empty.

There are 0 points in cube_141 / 3375
Cube_141 is empty.

There are 1 points in cube_142 / 3375
Found 1 clusters in cube_142

There are 0 points in cube_143 / 3375
Cube_143 is empty.

There are 0 points in cube_144 / 3375
Cube_144 is empty.

There are 0 points in cube_145 / 3375
Cube_145 is empty.

There are 0 points in cube_146 / 3375
Cube_146 is empty.

There are 0 points in cube_147 / 3375
Cube_147 is empty.

There are 0 points in cube_148 / 3375
Cube_148 is empty.

There are 0 points in cube_149 / 3375
Cube_149 is empty.

There are 0 points in cube_150 / 3375
Cube_150 is empty.

There are 0 points in cube_151 / 3375
Cube_151 is empty.

There are 0 points in cube_152 / 3375
Cube_152 is empty.

There are 0 points in cube_153 / 3375
Cube_153 is empty.

There are 0 points in cube_154 / 3375
Cube_154 is empty.

There are 0 points in cube_155 / 3375
Cube_155 is empty.

There are 0 points in cube_156 / 3375
Cube_156 is empty.

Ther

There are 0 points in cube_285 / 3375
Cube_285 is empty.

There are 0 points in cube_286 / 3375
Cube_286 is empty.

There are 0 points in cube_287 / 3375
Cube_287 is empty.

There are 0 points in cube_288 / 3375
Cube_288 is empty.

There are 0 points in cube_289 / 3375
Cube_289 is empty.

There are 0 points in cube_290 / 3375
Cube_290 is empty.

There are 0 points in cube_291 / 3375
Cube_291 is empty.

There are 0 points in cube_292 / 3375
Cube_292 is empty.

There are 0 points in cube_293 / 3375
Cube_293 is empty.

There are 0 points in cube_294 / 3375
Cube_294 is empty.

There are 0 points in cube_295 / 3375
Cube_295 is empty.

There are 0 points in cube_296 / 3375
Cube_296 is empty.

There are 0 points in cube_297 / 3375
Cube_297 is empty.

There are 0 points in cube_298 / 3375
Cube_298 is empty.

There are 0 points in cube_299 / 3375
Cube_299 is empty.

There are 2 points in cube_300 / 3375
Found 1 clusters in cube_300

There are 52 points in cube_301 / 3375
Found 1 clusters in cub

There are 0 points in cube_417 / 3375
Cube_417 is empty.

There are 0 points in cube_418 / 3375
Cube_418 is empty.

There are 0 points in cube_419 / 3375
Cube_419 is empty.

There are 0 points in cube_420 / 3375
Cube_420 is empty.

There are 0 points in cube_421 / 3375
Cube_421 is empty.

There are 0 points in cube_422 / 3375
Cube_422 is empty.

There are 0 points in cube_423 / 3375
Cube_423 is empty.

There are 0 points in cube_424 / 3375
Cube_424 is empty.

There are 0 points in cube_425 / 3375
Cube_425 is empty.

There are 0 points in cube_426 / 3375
Cube_426 is empty.

There are 0 points in cube_427 / 3375
Cube_427 is empty.

There are 0 points in cube_428 / 3375
Cube_428 is empty.

There are 0 points in cube_429 / 3375
Cube_429 is empty.

There are 0 points in cube_430 / 3375
Cube_430 is empty.

There are 0 points in cube_431 / 3375
Cube_431 is empty.

There are 0 points in cube_432 / 3375
Cube_432 is empty.

There are 0 points in cube_433 / 3375
Cube_433 is empty.

There are 0 po

...

There are 0 points in cube_2779 / 3375
Cube_2779 is empty.

There are 0 points in cube_2780 / 3375
Cube_2780 is empty.

There are 0 points in cube_2781 / 3375
Cube_2781 is empty.

There are 0 points in cube_2782 / 3375
Cube_2782 is empty.

There are 0 points in cube_2783 / 3375
Cube_2783 is empty.

There are 0 points in cube_2784 / 3375
Cube_2784 is empty.

There are 0 points in cube_2785 / 3375
Cube_2785 is empty.

There are 0 points in cube_2786 / 3375
Cube_2786 is empty.

There are 0 points in cube_2787 / 3375
Cube_2787 is empty.

There are 0 points in cube_2788 / 3375
Cube_2788 is empty.

There are 0 points in cube_2789 / 3375
Cube_2789 is empty.

There are 0 points in cube_2790 / 3375
Cube_2790 is empty.

There are 0 points in cube_2791 / 3375
Cube_2791 is empty.

There are 0 points in cube_2792 / 3375
Cube_2792 is empty.

There are 0 points in cube_2793 / 3375
Cube_2793 is empty.

There are 0 points in cube_2794 / 3375
Cube_2794 is empty.

There are 0 points in cube_2795 / 3375
C

There are 0 points in cube_2918 / 3375
Cube_2918 is empty.

There are 0 points in cube_2919 / 3375
Cube_2919 is empty.

There are 0 points in cube_2920 / 3375
Cube_2920 is empty.

There are 0 points in cube_2921 / 3375
Cube_2921 is empty.

There are 0 points in cube_2922 / 3375
Cube_2922 is empty.

There are 0 points in cube_2923 / 3375
Cube_2923 is empty.

There are 0 points in cube_2924 / 3375
Cube_2924 is empty.

There are 0 points in cube_2925 / 3375
Cube_2925 is empty.

There are 0 points in cube_2926 / 3375
Cube_2926 is empty.

There are 0 points in cube_2927 / 3375
Cube_2927 is empty.

There are 0 points in cube_2928 / 3375
Cube_2928 is empty.

There are 0 points in cube_2929 / 3375
Cube_2929 is empty.

There are 0 points in cube_2930 / 3375
Cube_2930 is empty.

There are 0 points in cube_2931 / 3375
Cube_2931 is empty.

There are 0 points in cube_2932 / 3375
Cube_2932 is empty.

There are 0 points in cube_2933 / 3375
Cube_2933 is empty.

There are 0 points in cube_2934 / 3375
C

There are 0 points in cube_3056 / 3375
Cube_3056 is empty.

There are 0 points in cube_3057 / 3375
Cube_3057 is empty.

There are 0 points in cube_3058 / 3375
Cube_3058 is empty.

There are 0 points in cube_3059 / 3375
Cube_3059 is empty.

There are 0 points in cube_3060 / 3375
Cube_3060 is empty.

There are 0 points in cube_3061 / 3375
Cube_3061 is empty.

There are 0 points in cube_3062 / 3375
Cube_3062 is empty.

There are 0 points in cube_3063 / 3375
Cube_3063 is empty.

There are 0 points in cube_3064 / 3375
Cube_3064 is empty.

There are 0 points in cube_3065 / 3375
Cube_3065 is empty.

There are 0 points in cube_3066 / 3375
Cube_3066 is empty.

There are 0 points in cube_3067 / 3375
Cube_3067 is empty.

There are 0 points in cube_3068 / 3375
Cube_3068 is empty.

There are 0 points in cube_3069 / 3375
Cube_3069 is empty.

There are 0 points in cube_3070 / 3375
Cube_3070 is empty.

There are 0 points in cube_3071 / 3375
Cube_3071 is empty.

There are 0 points in cube_3072 / 3375
C

There are 0 points in cube_3194 / 3375
Cube_3194 is empty.

There are 0 points in cube_3195 / 3375
Cube_3195 is empty.

There are 0 points in cube_3196 / 3375
Cube_3196 is empty.

There are 0 points in cube_3197 / 3375
Cube_3197 is empty.

There are 0 points in cube_3198 / 3375
Cube_3198 is empty.

There are 0 points in cube_3199 / 3375
Cube_3199 is empty.

There are 0 points in cube_3200 / 3375
Cube_3200 is empty.

There are 0 points in cube_3201 / 3375
Cube_3201 is empty.

There are 0 points in cube_3202 / 3375
Cube_3202 is empty.

There are 0 points in cube_3203 / 3375
Cube_3203 is empty.

There are 0 points in cube_3204 / 3375
Cube_3204 is empty.

There are 0 points in cube_3205 / 3375
Cube_3205 is empty.

There are 0 points in cube_3206 / 3375
Cube_3206 is empty.

There are 0 points in cube_3207 / 3375
Cube_3207 is empty.

There are 0 points in cube_3208 / 3375
Cube_3208 is empty.

There are 0 points in cube_3209 / 3375
Cube_3209 is empty.

There are 0 points in cube_3210 / 3375
C

There are 0 points in cube_3331 / 3375
Cube_3331 is empty.

There are 0 points in cube_3332 / 3375
Cube_3332 is empty.

There are 0 points in cube_3333 / 3375
Cube_3333 is empty.

There are 0 points in cube_3334 / 3375
Cube_3334 is empty.

There are 0 points in cube_3335 / 3375
Cube_3335 is empty.

There are 0 points in cube_3336 / 3375
Cube_3336 is empty.

There are 0 points in cube_3337 / 3375
Cube_3337 is empty.

There are 0 points in cube_3338 / 3375
Cube_3338 is empty.

There are 0 points in cube_3339 / 3375
Cube_3339 is empty.

There are 0 points in cube_3340 / 3375
Cube_3340 is empty.

There are 0 points in cube_3341 / 3375
Cube_3341 is empty.

There are 0 points in cube_3342 / 3375
Cube_3342 is empty.

There are 0 points in cube_3343 / 3375
Cube_3343 is empty.

There are 0 points in cube_3344 / 3375
Cube_3344 is empty.

There are 0 points in cube_3345 / 3375
Cube_3345 is empty.

There are 0 points in cube_3346 / 3375
Cube_3346 is empty.

There are 0 points in cube_3347 / 3375
C

### standard algorithm will most likely fill node color with red if there is any fraud transaction in it

In [14]:
colormap = np.ones_like(labels)
for node, indices in graph['nodes'].items():
    node_labels = mapper.data_from_cluster_id(node, graph, labels)
    #assert(np.allclose(ytrain[indices], node_labels))
    #print(fraud_mean)
    fraud_mean = 1.0
    if np.any(node_labels > 0):
        fraud_mean = 0.0
    #fraud_mean = (1.0 - (node_labels > 0).mean())**4
    #fraud_size = node_labels.size
    #tooltip_s.append((fraud_size, fraud_mean))
    colormap[indices] = fraud_mean

#tooltip_s = ytrain.copy()
#tooltip_s = np.array(tooltip_s)

### Tooltips will indicate if there are any false positives or true fraud

In [20]:
custom_tooltips = np.array(['']*len(labels), dtype=object)
custom_tooltips[labels == 0] = 'fp'
custom_tooltips[labels == 1] = 'FRAUD'

In [22]:
html = mapper.visualize(graph,
                 path_html="mean_pca_with_isof.html",
                 projected_X=proj_3d, color_function=colormap,
                 projected_X_names=['pca_mean_1', 'pca_mean_2', 'isof_score'], custom_tooltips = custom_tooltips,#10**(labels + 1),
                 title="mean_pca_with_isof(n_samples={})".format(len(baseamount_agg_mean_values)))

/home/const.belev/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Wrote visualization to: mean_pca_with_isof.html


In [13]:
display("mean_pca_with_isof.html")

In [12]:
!pwd

/home/yanina/TDA
